In [9]:
import pandas as pd
import numpy as np

In [5]:
master_data_test_transformed = pd.read_csv('master_data_test_transformed.csv')
master_data_test_transformed.head(2)

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,BUREAU_<LAMBDA>_Closed,BUREAU_<LAMBDA>_Sold
0,100001.0,0.0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241.0,-2329.0,-5170.0,...,0,0,0,0,0,0,1,0,1,0
1,100005.0,0.0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064.0,-4469.0,-9118.0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
master_data_train_transformed = pd.read_csv('master_data_train_transformed.csv')
master_data_train_transformed.head(2)

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_HOUSING_TYPE_With parents,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,BUREAU_<LAMBDA>_Closed,BUREAU_<LAMBDA>_Sold,TARGET
0,100002.0,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461.0,-637.0,-3648.0,...,0,0,0,0,0,0,1,1,0,1.0
1,100003.0,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,-1188.0,-1186.0,...,0,1,0,0,0,0,0,1,0,0.0


In [7]:
# Separate target from features
target = master_data_train_transformed['TARGET']
features_train = master_data_train_transformed.drop(columns=['TARGET'])
features_test = master_data_test_transformed

print("Train and test datasets loaded successfully.")

Train and test datasets loaded successfully.


In [10]:
# Check for missing or infinite values
print("Checking for NaN values in features_train:")
print(features_train.isnull().sum().sum())

print("\nChecking for infinite values in features_train:")
print(np.isinf(features_train.values).sum())

Checking for NaN values in features_train:
1144520

Checking for infinite values in features_train:
0


In [11]:
# Replace NaN values with column mean
features_train = features_train.fillna(features_train.mean())
# Confirm no missing or infinite values remain
print("Post-cleaning NaN values:", features_train.isnull().sum().sum())

Post-cleaning NaN values: 0


In [12]:
def calculate_vif(data, vif_threshold=10.0):
    vif_data = pd.DataFrame()
    vif_data["Feature"] = data.columns
    vif_data["VIF"] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]

    print("\nVIF Summary Before Filtering:")
    print(vif_data)

    # Filter features with VIF <= threshold
    selected_features = vif_data.loc[vif_data["VIF"] <= vif_threshold, "Feature"]
    reduced_data = data[selected_features]  # Filter only columns in the selected features
    return reduced_data, vif_data

In [14]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# Perform VIF calculation
features_train_vif, vif_summary = calculate_vif(features_train)

print("\nVIF Summary After Filtering:")
print(vif_summary[vif_summary["VIF"] <= 10.0])

# Save filtered features
features_test_vif = features_test[features_train_vif.columns]  # Apply to test dataset


VIF Summary Before Filtering:
                                  Feature           VIF
0                              SK_ID_CURR      1.000335
1                            CNT_CHILDREN  60488.825375
2                        AMT_INCOME_TOTAL      1.073672
3                              AMT_CREDIT     42.299122
4                             AMT_ANNUITY      2.889731
..                                    ...           ...
118   WEEKDAY_APPR_PROCESS_START_THURSDAY      1.676130
119    WEEKDAY_APPR_PROCESS_START_TUESDAY      1.709156
120  WEEKDAY_APPR_PROCESS_START_WEDNESDAY      1.689499
121                BUREAU_<LAMBDA>_Closed      1.139627
122                  BUREAU_<LAMBDA>_Sold      1.010063

[123 rows x 2 columns]

VIF Summary After Filtering:
                                  Feature       VIF
0                              SK_ID_CURR  1.000335
2                        AMT_INCOME_TOTAL  1.073672
4                             AMT_ANNUITY  2.889731
6              REGION_POPULATION_RE

In [16]:
from sklearn.feature_selection import mutual_info_classif

# Prepare data
features_train_filtered = features_train_vif.drop(columns=["SK_ID_CURR"])  # Drop ID column if not relevant
target_train = master_data_train_transformed["TARGET"]

# Calculate Information Gain
info_gain = mutual_info_classif(features_train_filtered, target_train, discrete_features=False)
ig_scores = pd.DataFrame({
    "Feature": features_train_filtered.columns,
    "Information Gain": info_gain
}).sort_values(by="Information Gain", ascending=False)

print("\nInformation Gain Scores:")
print(ig_scores)

# Select Top Features (e.g., top 20)
top_features = ig_scores.head(20)["Feature"].tolist()

# Filter the datasets
features_train_ig = features_train_filtered[top_features]
features_test_ig = features_test[top_features]  # Apply same features to test set

print(f"\nNumber of Features After IG Filtering: {len(top_features)}")
print(f"Top Features: {top_features}")


Information Gain Scores:
                       Feature  Information Gain
6                   FLAG_MOBIL          0.082099
8             FLAG_CONT_MOBILE          0.060377
73           FLAG_OWN_REALTY_Y          0.056414
80  NAME_FAMILY_STATUS_Married          0.054050
89      BUREAU_<LAMBDA>_Closed          0.047827
..                         ...               ...
34            FLAG_DOCUMENT_14          0.000000
37            FLAG_DOCUMENT_17          0.000000
40            FLAG_DOCUMENT_20          0.000000
43   AMT_REQ_CREDIT_BUREAU_DAY          0.000000
90        BUREAU_<LAMBDA>_Sold          0.000000

[91 rows x 2 columns]

Number of Features After IG Filtering: 20
Top Features: ['FLAG_MOBIL', 'FLAG_CONT_MOBILE', 'FLAG_OWN_REALTY_Y', 'NAME_FAMILY_STATUS_Married', 'BUREAU_<LAMBDA>_Closed', 'CODE_GENDER_M', 'FLAG_OWN_CAR_Y', 'EXT_SOURCE_3', 'FLAG_PHONE', 'EXT_SOURCE_2', 'AMT_ANNUITY', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'REG_CITY_NOT_WORK_CITY', 'FLAG_WORK_PHONE', 'POS_MAX', 'PREV_MAX', 

In [18]:
# Ensure train and test have the same features
assert set(features_train_ig.columns) == set(features_test_ig.columns), "Mismatch between train and test features!"

# Save the final filtered datasets
master_data_train_final = pd.concat([features_train_ig, target_train], axis=1)
master_data_test_final = features_test_ig  # No target column for test

# Save to files
master_data_train_final.to_csv("master_data_train_final.csv", index=False)
master_data_test_final.to_csv("master_data_test_final.csv", index=False)

print("Final datasets saved successfully!")

Final datasets saved successfully!


In [19]:
from google.colab import files

# Download the files
files.download("master_data_train_final.csv")
files.download("master_data_test_final.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>